In [9]:
import pandas as pd

In [10]:
# Read original annotation data
data = pd.read_excel('/vol/tensusers5/wharmsen/astla-data/dart-preposttest/data-in-different-formats/1-input-files/DART_test_data.xlsx')
data.loc[0,:]

Teacher                                                           31
Lijst                                                             16
Woord                                                           buik
Correct                                                            0
Set                                                               36
RecordingID                                                       16
file_path          /www/lst/live/writable/ybai_logs/exp2_03-02-20...
Time               Tue Mar 02 2021 10:20:17 GMT+0100 (Midden-Euro...
StudentID                                                       6111
SchoolID                                                           6
Consent1                                                           1
Consent2                                                           1
Leerlijn                                                        Maan
VerwijderdeLLn                                                     0
Condition                         

In [11]:
# Get names of audio files that all 51 teachers annotated
audioFileFrequencies = data['Audio recording'].value_counts()
mostAnnotatedAudioFiles = audioFileFrequencies[0:6].index
mostAnnotatedAudioFiles

Index(['https://gm.novolanguage.com/v0/sessions/309d1ae1-a4be-4ba7-a442-9f691b31eba5/audio/09eae7df-db7c-491b-b7c4-a7c9fcf62f47.mp3',
       'https://gm.novolanguage.com/v0/sessions/45fe2eff-918f-465d-a1a0-6e52c2471d87/audio/e5920085-8c1b-415c-b25d-24b5e5a4a2bf.mp3',
       'https://gm.novolanguage.com/v0/sessions/f5654c92-553c-43ea-a225-855ff92887f1/audio/aa809d8a-45d1-4cc0-a2c2-b40066d2849e.mp3',
       'https://gm.novolanguage.com/v0/sessions/c7d2e511-b40e-47dd-8a67-cae016a27553/audio/97549707-f715-4fe5-926c-b272b388e368.mp3',
       'https://gm.novolanguage.com/v0/sessions/912b7709-b51b-4a3d-bd43-13a7140fe60c/audio/74be41e8-d940-469f-8ebf-feb90250f9e8.mp3',
       'https://gm.novolanguage.com/v0/sessions/bd4da6ea-4986-4236-89b1-2ef95df2943a/audio/d52f2fbc-588a-4002-8c02-772dabdbfc16.mp3'],
      dtype='object')

In [12]:
# Check if there are 51 unique teacherIDs of each audio file.
for audioFile in mostAnnotatedAudioFiles:
    selection = data[data['Audio recording'] == audioFile]
    uniqueTeacherIDs = list(set(list(selection['Teacher'])))
    print(audioFile.split('/')[-1], len(uniqueTeacherIDs))

09eae7df-db7c-491b-b7c4-a7c9fcf62f47.mp3 51
e5920085-8c1b-415c-b25d-24b5e5a4a2bf.mp3 51
aa809d8a-45d1-4cc0-a2c2-b40066d2849e.mp3 51
97549707-f715-4fe5-926c-b272b388e368.mp3 51
74be41e8-d940-469f-8ebf-feb90250f9e8.mp3 51
d52f2fbc-588a-4002-8c02-772dabdbfc16.mp3 51


In [13]:
#Print specifics of audio file
output = []
for audioFile in mostAnnotatedAudioFiles:
    selection = data[data['Audio recording'] == audioFile].reset_index()
    kern = selection.loc[0, 'Kern']
    exerciseID = selection.loc[0, 'ExerciseID']
    test = kern + str(exerciseID)
    studentID = selection.loc[0, 'StudentID']
    nrUniqueTeachers = len(list(set(list(selection['Teacher']))))
    nrUniqueWords = len(list(set(list(selection['Woord']))))
    output.append([audioFile.split('/')[-1], kern, exerciseID, test, studentID, nrUniqueTeachers, nrUniqueWords])

audioMetaData = pd.DataFrame(output, columns=['audio', 'kern', 'exerciseID', 'test', 'speaker', 'nrOfAnnotations', 'nrOfWords'])
audioMetaData

,audio,kern,exerciseID,test,speaker,nrOfAnnotations,nrOfWords
0,09eae7df-db7c-491b-b7c4-a7c9fcf62f47.mp3,pretest,2,pretest2,33107,51,24
1,e5920085-8c1b-415c-b25d-24b5e5a4a2bf.mp3,posttest,1,posttest1,4121,51,24
2,aa809d8a-45d1-4cc0-a2c2-b40066d2849e.mp3,pretest,0,pretest0,13105,51,24
3,97549707-f715-4fe5-926c-b272b388e368.mp3,posttest,0,posttest0,6122,51,24
4,74be41e8-d940-469f-8ebf-feb90250f9e8.mp3,pretest,1,pretest1,6101,51,24
5,d52f2fbc-588a-4002-8c02-772dabdbfc16.mp3,posttest,2,posttest2,1208,51,24


In [14]:
# Read prompts
references = pd.read_csv("../../astla-data/dart-preposttest/data-in-different-formats/1-input-files/reference_wordlist.csv", sep=";")
references = references.set_index('name')
references

,prompt
name,
pretest0,buik web lift grapje spierkracht bal sterk mee...
pretest1,pauw sterren gat proost lijn vlo kijken lus ba...
pretest2,jong lach strik vuur specht keelpijn sla buite...
posttest0,draad strik bloemen meetlat schroef waarde spr...
posttest1,lesje kijken hagel Jaap sterk bestaat dure den...
posttest2,vlo mooi trommel stoep nicht onze slepen raamp...


In [24]:
allAssessmentDF = pd.DataFrame()
for audioFile in mostAnnotatedAudioFiles:
    selection = data[data['Audio recording'] == audioFile].reset_index()
    uniqueTeachers = list(set(list(selection['Teacher'])))

    # Get Test Name
    audio = audioFile.split('/')[-1]
    testName = audioMetaData.set_index('audio').loc[audio,'test']

    # Get prompt wordlist
    uniqueWords = references.loc[testName, 'prompt'].split(' ')
    index = [audio.replace('.mp3', '') + '_' + word for word in uniqueWords]

    # Create dataframe to save teacher judgements
    assessmentDF = pd.DataFrame(columns = list(set(list(selection['Teacher']))), 
                   index = index)
    
    for idx,row in selection.iterrows():
        teacherID = row['Teacher']
        index = audio.replace('.mp3', '') + '_' + row['Woord']
        assessment = row['Correct']

        assessmentDF.loc[index, teacherID] = assessment

    assessmentDF.to_csv('../../astla-data/dart-preposttest/slate-data/1-teacherAssessments/' + audio.replace('.mp3', '') + '_' + testName + '_teacherAssessments.csv', sep=',', encoding='utf-8')
    allAssessmentDF = pd.concat([allAssessmentDF, assessmentDF])

allAssessmentDF.to_csv('../../astla-data/dart-preposttest/slate-data/1-teacherAssessments/' + 'all_teacherAssessments.csv', sep=',', encoding='utf-8')
allAssessmentDF


,1,2,3,4,6,7,8,9,10,11,...,44,47,48,49,50,51,52,53,54,55
09eae7df-db7c-491b-b7c4-a7c9fcf62f47_jong,0,1,0,1,0,1,1,1,1,1,...,1,0,1,0,1,1,0,1,1,1
09eae7df-db7c-491b-b7c4-a7c9fcf62f47_lach,1,0,1,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
09eae7df-db7c-491b-b7c4-a7c9fcf62f47_strik,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
09eae7df-db7c-491b-b7c4-a7c9fcf62f47_vuur,1,1,1,1,0,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
09eae7df-db7c-491b-b7c4-a7c9fcf62f47_specht,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d52f2fbc-588a-4002-8c02-772dabdbfc16_vondst,1,0,1,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
d52f2fbc-588a-4002-8c02-772dabdbfc16_bank,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
d52f2fbc-588a-4002-8c02-772dabdbfc16_markt,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
d52f2fbc-588a-4002-8c02-772dabdbfc16_spuw,0,1,0,1,0,1,1,1,0,1,...,1,0,1,1,1,1,0,1,1,0
